## Configuration and Setup


In [1]:
import os
os.environ['HF_HOME'] = "C:/HF_CACHE/"

In [2]:
import torch

from cs324_project.datasets import GlueDatasetTask, load_glue_dataset_info
from cs324_project.models import ModelCheckpointName, load_classification_model, load_pretraining_model, load_tokenizer
from cs324_project.masking import (
    get_training_args_mlm, get_trainer_mlm, RandomMaskingConfig, WholeWordMaskingConfig, TyphoonMaskingConfig)
from cs324_project.classification import get_training_args_sc, get_trainer_sc

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
model_name = ModelCheckpointName.TINYBERT_HUAWEI
task = GlueDatasetTask.MRPC

In [5]:
tokenizer = load_tokenizer(model_name)
dataset_info = load_glue_dataset_info(task, tokenizer)

Found cached dataset glue (C:/HF_CACHE/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-210d87aa331c9a9e.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-33abd690c2474663.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-fd99a719fa52af0a.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-98cf444ee3350795.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-9383850b1455b190.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-bcdcf9ec1f3ace0e.arro

## Fine-tune with masking

In [6]:
model_mlm = load_pretraining_model(model_name, dataset_info)

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForMaskedLM: ['fit_denses.4.bias', 'fit_denses.2.bias', 'fit_denses.1.bias', 'fit_denses.1.weight', 'cls.seq_relationship.weight', 'fit_denses.3.weight', 'fit_denses.0.bias', 'fit_denses.2.weight', 'fit_denses.0.weight', 'fit_denses.3.bias', 'fit_denses.4.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
training_args_mlm = get_training_args_mlm(
    masking_config=RandomMaskingConfig(),
    num_epochs=100)
trainer_mlm = get_trainer_mlm(
    dataset_info=dataset_info,
    mlm_args=training_args_mlm,
    model=model_mlm)

Creating training arguments, model output dir: C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-21-2023 10-10-55 AM


In [8]:
trainer_mlm.train()

C:\Users\Windows\anaconda3\envs\cs324\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,6.093100,5.315406
2,5.012200,4.694579
3,4.548800,4.324155
4,4.218300,4.110555
5,3.993600,3.946275
6,3.818500,3.795141
7,3.663600,3.653197
8,3.512000,3.588183
9,3.446500,3.577208
10,3.314600,3.504378


TrainOutput(global_step=23000, training_loss=2.2030989287003226, metrics={'train_runtime': 1700.288, 'train_samples_per_second': 215.728, 'train_steps_per_second': 13.527, 'total_flos': 786121535232432.0, 'train_loss': 2.2030989287003226, 'epoch': 100.0})

In [9]:
trainer_mlm.evaluate()

{'eval_loss': 3.0934484004974365,
 'eval_runtime': 0.6588,
 'eval_samples_per_second': 619.352,
 'eval_steps_per_second': 39.469,
 'epoch': 100.0}

In [10]:
print(trainer_mlm.state.best_model_checkpoint)

C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-21-2023 10-10-55 AM\checkpoint-13110


## Fine-tune on sequence classification task

In [35]:
model_sc = load_classification_model(trainer_mlm.state.best_model_checkpoint, dataset_info)

Some weights of the model checkpoint at C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-21-2023 10-10-55 AM\checkpoint-13110 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequ

In [36]:
training_args_sc = get_training_args_sc(
    task,
    num_epochs=100)
trainer_sc = get_trainer_sc(
    dataset_info=dataset_info,
    model=model_sc,
    training_args=training_args_sc)

Creating training arguments, model output dir: C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\sc\Model 03-21-2023 10-47-24 AM


In [37]:
trainer_sc.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.680700,0.657122,0.683824,0.812227
2,0.645800,0.625719,0.683824,0.812227
3,0.624600,0.611216,0.683824,0.812227
4,0.613500,0.602546,0.683824,0.812227
5,0.602700,0.593513,0.683824,0.812227
6,0.591500,0.581803,0.683824,0.812227
7,0.580900,0.571933,0.691176,0.815789
8,0.568900,0.563185,0.703431,0.820208
9,0.560000,0.555053,0.705882,0.819277
10,0.551300,0.546830,0.713235,0.821374


TrainOutput(global_step=23000, training_loss=0.3931417942876401, metrics={'train_runtime': 907.1687, 'train_samples_per_second': 404.335, 'train_steps_per_second': 25.354, 'total_flos': 781014765977712.0, 'train_loss': 0.3931417942876401, 'epoch': 100.0})

In [38]:
trainer_sc.evaluate()

{'eval_loss': 0.5205249190330505,
 'eval_accuracy': 0.7475490196078431,
 'eval_f1': 0.8170515097690941,
 'eval_runtime': 0.4713,
 'eval_samples_per_second': 865.726,
 'eval_steps_per_second': 55.169,
 'epoch': 100.0}

In [39]:
print(trainer_sc.state.best_model_checkpoint)

C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\sc\Model 03-21-2023 10-47-24 AM\checkpoint-3910
